In [2]:
import pandas as pd

data = pd.read_csv("Task 3 and 4_Loan_Data.csv")
data.head(10)

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
5,4661159,0,5376.886873,7189.121298,85529.84591,2,697,0
6,8291909,1,3634.057471,7085.980095,68691.57707,6,722,0
7,4616950,4,3302.172238,13067.570210,50352.16821,3,545,1
8,3395789,0,2938.325123,1918.404472,53497.37754,4,676,0
9,4045948,0,5396.366774,5298.824524,92349.55399,2,447,0


In [3]:
# In this task we are tasked with quantization of FICO scores to class labels, and we need to find the most appropriate
# bucket boundaries.

# For simplicity lets only consider intervals of 10. We are going to seek to maximize the log likelihood of the dataset given
# LL (b_1.... b_r-1) = Summation from 1 to r of k_i ln (p_i) + (n_i - k_i ) ln(1 - p_i), where b_i are bucket boundaries,
# n_i is number of records in bucket, k_i is # of defaults, and p_i = k_i / n_i is % of defaults in bucket


# We can seek to solve this via a dp approach. Let us define the transition state dp[i][j][n] as the max log-likelihood of
# The fisco range from [i,j] with n buckets. We see that dp[i][j][1] is the base case of applied formula, and
# dp[i][j][n] = argmax_(k,p,q) (dp[i][k][p] + dp[k+1][j]) where p+j = n, and k exists in [i,j-1].

#Lets first get the array consisting of the pair (fico_score, default) and sort it

fico_scores = []

for i in range(len(data['fico_score'])):
    fico_scores.append((data['fico_score'][i],data['default'][i]))


fico_scores.sort()
#print(fico_scores)

defaults = [entry[1] for entry in fico_scores]
fico_scores = [entry[0] for entry in fico_scores]

print(defaults)
print(fico_scores)

prefdefaults = [0 for _ in range(len(defaults))]
prefdefaults[0]=defaults[0]

for i in range(1,len(defaults)):
    if defaults[i]==1:
        prefdefaults[i]=prefdefaults[i-1]+1
    else:
        prefdefaults[i]=prefdefaults[i-1]
# here we can see range lies from 400 - 800

[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 

In [4]:
# Lets initialize the dp array. Let's consider buckets only in intervals of fico scores of 10. Then we have
# 45 maximum bucket slots.
import math
from math import log



# Returns index of first element that compares >=value
def lower_bound(arr, value):
    left, right = 0, len(arr) - 1
    result = -1

    while left <= right:
        mid = (left + right) // 2

        if arr[mid] >= value:
            result = mid
            right = mid - 1
        else:
            left = mid + 1
    return result

#Returns first element greater than some value
def upper_bound(arr, value):
    left, right = 0, len(arr)-1
    result= -1

    while left<=right:
        mid = (left+right)//2
        if arr[mid]>value:
            result = mid
            right = mid-1
        else:
            left = mid+1
    return result

ERROR = -100000000


def solve(i,j,buckets,dp,fico_scores,prefdefaults,dpbuckets):
    global ERROR
    if dp[i][j][buckets]!=-1:
        return dp[i][j][buckets]
    if buckets == 1:
        # We are considering the range from [i*10, j*10)
        # compute log likelihood of sub bucket
        # Lets use binary search to find what we are looking for
        dpbuckets[i][j][buckets]= [i*10,j*10]
        left = lower_bound(fico_scores, i*10)
        right = upper_bound(fico_scores,j*10-1)
        n = right-left
        if n==0:
            dp[i][j][buckets]=0
            return dp[i][j][buckets]
        k = 0
        if right>0:
            k+=prefdefaults[right-1]
        if left>0:
            k-=prefdefaults[left-1]
        p = k/n
        # avoid log domain imbalance with smoothign
        if p==0:
            p+=0.00001
        elif p==1:
            p-=0.00001
        dp[i][j][buckets]= k*log(p)+(n-k)*log(1-p)
        return dp[i][j][buckets]

    if j-i<buckets:
        #Degenerage case, not enough space to make n buckets:
        dp[i][j][buckets]=ERROR
        return dp[i][j][buckets]

    # Otherwise, we need to consider splits across all possible intermediates

    ind_middle = -1
    bucket_middle = -1
    for split in range(i+1,j):
        for m in range(1,buckets):
            left = solve(i,split,m,dp,fico_scores,prefdefaults,dpbuckets)
            right = solve(split,j,buckets-m,dp,fico_scores,prefdefaults,dpbuckets)
            if left!=ERROR and right!=ERROR:
                if left+right>dp[i][j][buckets] or dp[i][j][buckets]==-1:
                    # remember [i,m] and [m,j] as the max bukets so far
                    dp[i][j][buckets]=left+right
                    ind_middle = split
                    bucket_middle = m
    dpbuckets[i][j][buckets] = dpbuckets[i][ind_middle][bucket_middle] + dpbuckets[ind_middle][j][buckets-bucket_middle][1:]
    return dp[i][j][buckets]

dp = [[[-1 for _ in range(16)] for _ in range(86)] for _ in range(86)]

# Keeps track of the boundaries for the buckets for each dp state
dpbuckets = [[[[]for _ in range(16)] for _ in range(86)] for _ in range(86)]
for i in range(86):
    for j in range(86):
        for k in range(16):
            solve(i,j,k,dp,fico_scores,prefdefaults,dpbuckets)

print(dp[35][85])
print(dpbuckets[35][85])


[-1, -4790.189189997091, -4425.623496938375, -4319.412685738522, -4282.800089370939, -4260.2989696218, -4244.495932021195, -4236.767594113566, -4233.100795214499, -4230.605845655421, -4228.17978331659, -4227.106435650329, -4226.383641043643, -4225.646159929748, -4224.572812263487, -4223.954127693484]
[[], [350, 850], [350, 610, 850], [350, 580, 650, 850], [350, 520, 580, 650, 850], [350, 520, 580, 640, 700, 850], [350, 520, 580, 610, 650, 720, 850], [350, 520, 550, 580, 610, 650, 720, 850], [350, 520, 550, 580, 610, 640, 660, 720, 850], [350, 500, 520, 550, 580, 610, 640, 660, 720, 850], [350, 500, 520, 550, 580, 610, 640, 650, 700, 720, 850], [350, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 850], [350, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 730, 850], [350, 500, 520, 550, 580, 610, 640, 650, 700, 720, 790, 810, 820, 850], [350, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 790, 810, 820, 850], [350, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 730, 790, 810,

In [6]:
print(dp[40][85])
print(dpbuckets[40][85])

[-1, -4790.189189997091, -4425.623496938375, -4319.412685738522, -4282.800089370939, -4260.2989696218, -4244.495932021195, -4236.767594113566, -4233.100795214499, -4230.605845655421, -4228.17978331659, -4227.106435650329, -4226.383641043643, -4225.646159929748, -4224.572812263487, -4223.954127693484]
[[], [400, 850], [400, 610, 850], [400, 580, 650, 850], [400, 520, 580, 650, 850], [400, 520, 580, 640, 700, 850], [400, 520, 580, 610, 650, 720, 850], [400, 520, 550, 580, 610, 650, 720, 850], [400, 520, 550, 580, 610, 640, 660, 720, 850], [400, 500, 520, 550, 580, 610, 640, 660, 720, 850], [400, 500, 520, 550, 580, 610, 640, 650, 700, 720, 850], [400, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 850], [400, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 730, 850], [400, 500, 520, 550, 580, 610, 640, 650, 700, 720, 790, 810, 820, 850], [400, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 790, 810, 820, 850], [400, 500, 520, 550, 580, 610, 640, 650, 660, 700, 720, 730, 790, 810,